In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [14]:
def split_scalar(x_indep,y_dep):
    x_train, x_test, y_train, y_test = train_test_split(x_indep, y_dep, test_size = 0.25, random_state = 0)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)    
    return x_train, x_test, y_train, y_test

def r2_prediction(regressor,x_test,y_test):
    y_pred = regressor.predict(x_test)
    r2=r2_score(y_test,y_pred)
    return r2

def Linear(x_train,y_train,x_test,y_test):       
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return  r2

def svm_linear(x_train,y_train,x_test,y_test):         
    from sklearn.svm import SVR
    regressor = SVR(kernel = 'linear')
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return r2  
    
def svm_NL(x_train,y_train,x_test,y_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel = 'rbf')
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return r2  
     

def Decision(x_train,y_train,x_test,y_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state = 0)
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return r2  
     
def random(x_train,y_train,x_test,y_test):       
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
    regressor.fit(x_train, y_train)
    r2=r2_prediction(regressor,x_test,y_test)
    return r2

In [36]:
def rfeFeature(x_indep,y_dep,n):
    rfeList=[]
    models=[
        LogisticRegression(solver='lbfgs',max_iter=2000),
        RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0),
        DecisionTreeClassifier(criterion='gini',max_features='sqrt',splitter='best',random_state=0),
        SVC(kernel='linear',random_state=0)
    ]

    for model in models:
        rfe=RFE(estimator=model, n_features_to_select=n)
        transformed = rfe.fit_transform(x_indep, y_dep)
        rfeList.append(transformed)
    return rfeList

def rfe_regression(acclin,accsvml,accsvmnl,accdes,accrf): 
    rfedataframe=pd.DataFrame(index=['Linear','SVC','Random','DecisionTree'],columns=['Linear','SVML','SVMNL','Decision','Random'])
    for number,idex in enumerate(rfedataframe.index):
        rfedataframe['Linear'][idex]=acclin[number]
        rfedataframe['SVML'][idex]=accsvml[number]
        rfedataframe['SVMNL'][idex]=accsvmnl[number]
        rfedataframe['Decision'][idex]=accdes[number]
        rfedataframe['Random'][idex]=accrf[number]
        
    return rfedataframe

In [37]:
dataset=pd.read_csv("prep.csv")
dataset=pd.get_dummies(dataset,drop_first=True)

x_indep=dataset.drop('classification_yes',axis=1)
y_dep=dataset['classification_yes']

rfelist=rfeFeature(x_indep,y_dep,3)
acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]

for i in rfelist:
    x_train, x_test, y_train, y_test=split_scalar(i,y_dep)
    
    r2_lin=Linear(x_train,y_train,x_test,y_test)
    acclin.append(r2_lin)
    
    r2_sl=svm_linear(x_train,y_train,x_test,y_test)    
    accsvml.append(r2_sl)
    
    r2_NL=svm_NL(x_train,y_train,x_test,y_test)
    accsvmnl.append(r2_NL)

    r2_d=Decision(x_train,y_train,x_test,y_test)
    accdes.append(r2_d)
    
    r2_r=random(x_train,y_train,x_test,y_test)
    accrf.append(r2_r)
    
result=rfe_regression(acclin,accsvml,accsvmnl,accdes,accrf)
result

,Linear,SVML,SVMNL,Decision,Random
Linear,0.601845,0.415994,0.74825,0.775644,0.774526
SVC,0.571731,0.548349,0.739797,0.680507,0.715455
Random,0.580359,0.536163,0.819616,0.852326,0.824784
DecisionTree,0.416838,0.162207,0.505252,0.583884,0.584428
